In [1]:
import cv2
import matplotlib.pyplot as plt

# Hàm đọc keypoints từ file .pts

# Hàm để load ảnh và vẽ keypoints lên
def plot_keypoints(image_path, pts_file):
    # Đọc ảnh
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Chuyển thành RGB cho phù hợp với Matplotlib

    # Đọc keypoints
    keypoints = read_keypoints(pts_file)

    # Hiển thị ảnh và các keypoints
    plt.imshow(img)
    for (x, y) in keypoints:
        plt.scatter([x], [y], c='red', s=10)
    
    plt.show()


In [2]:
import os
import cv2
import torch
from torch.utils.data import Dataset

class FaceLandmarkDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        """
        image_dir: Đường dẫn đến folder chứa ảnh .jpg và file .pts
        transform: Bất kỳ phép biến đổi nào (augmentation, normalization, ...)
        """
        self.image_dir = image_dir
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        # Lấy tên file ảnh
        image_file = self.image_files[idx]
        # Tương ứng file .pts
        pts_file = image_file.replace('.jpg', '.pts')
        
        # Đọc ảnh
        image_path = os.path.join(self.image_dir, image_file)
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Không thể đọc ảnh từ {image_path}. Đảm bảo rằng đường dẫn là chính xác và file tồn tại.")
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Chuyển sang RGB

        # Đọc tọa độ landmark từ file .pts
        pts_path = os.path.join(self.image_dir, pts_file)
        landmarks = self.read_pts(pts_path)
        if landmarks is None:
            raise ValueError(f"Không thể đọc keypoints từ {pts_path}. Đảm bảo rằng file tồn tại và định dạng đúng.")
        
        # Chuyển đổi landmarks thành numpy array và tensor
        landmarks = torch.tensor(landmarks, dtype=torch.float32)

        # Áp dụng các phép biến đổi (nếu có)
        if self.transform:
            transformed = self.transform(image=image, landmarks=landmarks)
            image = transformed['image']
            landmarks = transformed['landmarks']
        else:
            # Chuyển đổi numpy sang tensor và normalize ảnh
            image = torch.from_numpy(image).permute(2, 0, 1).float() / 255.0  # (C, H, W)

        return image, landmarks

    def read_pts(self, pts_file):
        keypoints = []
        if not os.path.isfile(pts_file):
            return None
        with open(pts_file, 'r') as f:
            lines = f.readlines()[3:-1]  # Bỏ qua 3 dòng đầu và dòng cuối
            for line in lines:
                try:
                    x, y = map(float, line.strip().split())
                    keypoints.append((x, y))
                except ValueError:
                    return None
        return keypoints

# Ví dụ sử dụng
dataset = FaceLandmarkDataset(image_dir='D:\lmvh\lmvh_proj_filter\src\data\dataresized')
image, landmarks = dataset[0]

print(image.shape)       # Tensor của ảnh: (C, H, W)
print(landmarks.shape)   # Tensor của landmarks: (68, 2)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'D:/Tự học/lmvh-proj-filter/src/data/dataresized'